# BMIN 5200 Foundations of Artificial Intelligence in Health

Assignment 3: Expert Systems in clipspy

In [18]:
import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

In [19]:
# create the CLIPS environment
env = clips.Environment()

# Endometriosis Classification System

Endometriosis Classification Algorithm:
Using clipspy to implement an expert system that classifies a patient into one of the following categories:

Outputs:
1. Endometriosis
2. Not Endometriosis (some other disease causing symptoms)
3. Endometriosis and another concomitant disease

![resource](reference/clinical_diagnosis_algo.png)

# Templates

## Part 1: Defining Fact Templates (that are consistent with endometriosis)

This section defines and builds all of the templates that are needed to track and evalute facts associated with inclusion criteria.

In [20]:
DEFTEMPLATE_PATIENT_ENDOMETRIOSIS_SYMPTOMS = """
(deftemplate patient_endo_symptoms
    (slot abdominal_pelvic_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot dysmenorrhea (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot pain_with_sex (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot dyschezia (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot dysuria (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot infertility (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot pelvic_perineal_pain (type SYMBOL)
        (allowed-symbols yes no unknown))

    )
"""
env.build(DEFTEMPLATE_PATIENT_ENDOMETRIOSIS_SYMPTOMS)

# patient concomitant disease symptoms
DEFTEMPLATE_PATIENT_CONCOMITANT_DISEASE_SYMPTOMS = """
(deftemplate patient_concomitant_disease_symptoms
    (slot amenorrhea (type SYMBOL) 
        (allowed-symbols yes no unknown))
    (slot constipation (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot diarrhea (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot flank_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot hematuria (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot frequent_urination (type SYMBOL)
        (allowed-symbols yes no unknown))
    (slot adenomyosis (type SYMBOL)
        (allowed-symbols yes no unknown))

)
"""
env.build(DEFTEMPLATE_PATIENT_CONCOMITANT_DISEASE_SYMPTOMS)


## Part 2: Defining Fact Templates (factors indicating to consider another diagnossis in addition to OR instead of endometriosis)
Consistent with / not consistent wtih endo Fact Templates

## Consistent with / not consistent with endo Fact Templates
This section defines and builds all of the templates that are needed to track and evalute whether inclusion, exclusion, criteria for endometriosis and/or concomitant diseases.

In [21]:
# participant inclusion criteria met status
DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION = """
(deftemplate endometriosis_inclusion
    (slot meets_criteria (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION)

# participant trial eligibility status
DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION = """
(deftemplate concomitant_disease_inclusion
    (slot meets_criteria (type SYMBOL)
    (allowed-symbols yes no unknown possible))
)
"""
env.build(DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION)

# Initialize Knowledge Base

This section adds the necessary `deffacts`. These include four facts with slot values set to `unknown` to prevent some rules from firing at inappropriate times. As different rules are executed and facts asserted, these slot values will be updated so that, through chaining, other rules will then fire as needed. You should review the already implemented `defrules` carefully to understand how these work.

In [22]:
# Add deffacts that the inclusion, exclusion, trial eligibility, and child bearing status are all unknown
DEFFCATS_INITIAL_STATUS = """
(deffacts starting_inclusion_exclusion_facts "Set the inclusion criteria met to unknown"
    (endometriosis_inclusion (meets_criteria unknown))
    (concomitant_disease_inclusion (meets_criteria unknown))
)
"""
env.build(DEFFCATS_INITIAL_STATUS)

# reset the environment to make sure the deffacts are added
env.reset()

# Inference Rules

## Inclusion Criteria Rules

The `defrules` in this section support gathering user input about the patient relative to the inclusion criteria and determining if the inclusion criteria are met. Notice that if the initial criteria (age, Karnofsky quality of life, life expectancy, and HIV status) are not met, no further information is collected as it is already known that trial eligibility will not be satisfied. If the initial inclusion criteria are met, then the value of the patient's sex is requested. If the patient is female, yet more information is requested.

In [33]:
# # These are the prompts that will be displayed to the user when requesting various template:slot inputs.
# prompt_map = {
#     "patient:name_is": "Enter patient name: ",
#     "patient:age_is": "Enter patient age (in years): ",
#     "pelvic_pain:has_pain": "Does the patient have acute pelvic pain (yes or no or unknown)?: ",
#     "dysmenorrhea:has_dysmenorrhea": "Does the patient have dysmenorrhea (yes or no or unknown)?: ",
#     "dyspareunia:has_dyspareunia": "Does the patient have dyspareunia (yes or no or unknown)?: ",
#     "dyschezia:has_dyschezia" : "Does the patient have dyschezia (yes or no or unknown)?: ",
#     "dysuria:has_dysuria" : "Does the patient have dysuria (yes or no or unknown)?: ",
#     "infertility:is_infertile" : "Is the patient infertile (yes or no or unknown)?: ",
#     "chronic_pelvic_pain:has_chronic_pelvic_pain" : "Does the patient have chronic pelvic pain (peritonitis) (yes or no or unknown)?: ",
#     "amenorrhea:has_amenorrhea" : "Does the patient have amenorrhea (yes or no or unknown)?: ",
#     "constipation:has_constipation" : "Does the patient have constipation related to IBS (yes or no or unknown)?: ",
#     "diarrhea:has_diarrhea" : "Does the patient have diarrhea (yes or no or unknown)?: ",
#     "flank_pain:has_flank_pain" : "Does the patient have flank pain (yes or no or unknown)?: ",
#     "hematuria:has_hematuria" : "Does the patient have hematuria (yes or no or unknown)?: ",
#     "frequent_urination:has_frequent_urination" : "Does the patient have frequent urination (yes or no or unknonw)?: ", 
#     "adenomyosis:has_adenomyosis" : "Does the patient have adenomyosis (yes or no or unknown)?: "
# }
# build_read_assert(env, prompt_map) # see src/clips_util.py for implementation and demos/clips/clips_basics.ipynb for usage

# DEFRULE_READ_INITIAL_INCLUSION = """
# (defrule reading_input_initial
#     =>
#     (read_assert patient_endo_symptoms)
#     (read_assert patient_concomitant_disease_symptoms)
# )
# """
# env.build(DEFRULE_READ_INITIAL_INCLUSION)

# ; RULE: Inclusion Criteria Are Not Met
# ; *Forward chaining to determine if participant is not eligible for study based on inclusion criteria facts
# ; INPUT: Criteria based on inclusion criteria defined in study. 
# ; OUTPUT: Trial eligibility No, Inclusion Criteria Met No
DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_NOT_MET = """
(defrule inclusion-criteria-notmet "Rule to define a person as not eligible for study based on inclusion criteria facts"
    (logical
        (and
            (patient_endo_symptoms (abdominal_pelvic_pain ~yes))  
            (patient_endo_symptoms (dysmenorrhea ~yes))   
            (patient_endo_symptoms (pain_with_sex ~yes)) 
            (patient_endo_symptoms (dyschezia ~yes)) 
            (patient_endo_symptoms (dysuria ~yes)) 
            (patient_endo_symptoms (infertility ~yes))   
            (patient_endo_symptoms (pelvic_perineal_pain ~yes))
        )
    )

    ?f1 <-(endometriosis_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria no))
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_NOT_MET)

# indicates presence of symptom(s) that are consistent with endometriosis
DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_MET = """
(defrule inclusion-criteria-met "Rule to define a person as having symptom(s) consistent with endometriosis based on inclusion criteria facts"
    (logical
        (or
            (patient_endo_symptoms (abdominal_pelvic_pain yes))  
            (patient_endo_symptoms (dysmenorrhea yes))   
            (patient_endo_symptoms (pain_with_sex yes)) 
            (patient_endo_symptoms (dyschezia yes)) 
            (patient_endo_symptoms (dysuria yes)) 
            (patient_endo_symptoms (infertility yes))   
            (patient_endo_symptoms (pelvic_perineal_pain yes))
        )
    )

    ?f1 <-(endometriosis_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria yes))
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_MET)

# indicates presence of symptoms that indicate no concomitant disease (along with or instead of) endometriosis
DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_NOT_MET = """
(defrule inclusion-criteria-not-met "Rule to define a person as having symptom(s) not consistent with other (non-endo) disease based on inclusion criteria facts"
    (logical
        (and
            (patient_concomitant_disease_symptoms (amenorrhea ~yes))  
            (patient_concomitant_disease_symptoms (constipation ~yes))   
            (patient_concomitant_disease_symptoms (diarrhea ~yes)) 
            (patient_concomitant_disease_symptoms (flank_pain ~yes)) 
            (patient_concomitant_disease_symptoms (hematuria ~yes)) 
            (patient_concomitant_disease_symptoms (frequent_urination ~yes))   
            (patient_concomitant_disease_symptoms (adenomyosis ~yes))
        )
    )

    ?f1 <-(concomitant_disease_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria no))
)
"""
env.build(DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_NOT_MET)

# indicates presence of symptoms that indicate concomitant disease (along with or instead of) endometriosis
DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_MET = """
(defrule inclusion-criteria-met "Rule to define a person as having symptom(s) consistent with other (non-endo) disease based on inclusion criteria facts"
    (logical
        (or
            (patient_concomitant_disease_symptoms (amenorrhea yes))  
            (patient_concomitant_disease_symptoms (constipation yes))   
            (patient_concomitant_disease_symptoms (diarrhea yes)) 
            (patient_concomitant_disease_symptoms (flank_pain yes)) 
            (patient_concomitant_disease_symptoms (hematuria yes)) 
            (patient_concomitant_disease_symptoms (frequent_urination yes))   
            (patient_concomitant_disease_symptoms (adenomyosis yes))
        )
    )

    ?f1 <-(concomitant_disease_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria yes))
)
"""
env.build(DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_MET)
# not going to try and classify the concomitant disease - but will factor whether they do have a concomitant disease (IBS, or interstitial cystitis, or adenomyosis) in my evaluation
    # because I don't have detailed symptom data for all of the concomitant diseases, only some of them


## Inclusion Criteria Status Reporting Rules

The cell below defines `defrules` that fire once it is determine if the inclusion criteria are satisfied or not to report the status to the user.

In [ ]:
evaluation_dict = {'only_endo_predicted':0, 'only_endo_actual':0, 'only_concomitant_predicted':0, 'only_concomitant_actual':0, 'both_endo_concomitant_predicted':0, 'both_endo_concomitant_actual':0, 'neither_endo_concomitant_predicted':0, 'neither_endo_concomitant_actual':0}

def store_result(*args):
    endo, other = args
    if endo == "endo_0" and other == "other_0":
        evaluation_dict['neither_endo_concomitant_predicted'] += 1
    elif endo == "endo_0" and other == "other_1":
        evaluation_dict['only_concomitant_predicted'] += 1
    elif endo == "endo_1" and other == "other_0":
        evaluation_dict['only_endo_predicted'] += 1
    elif endo == "endo_1" and other == "other_1":
        evaluation_dict['both_endo_concomitant_predicted'] += 1

env.define_function(store_result)

In [ ]:
DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_INCLUSION = """
(defrule endo-and-concoomitant-inclusion
    (endometriosis_inclusion (meets_criteria yes))
    (concomitant_disease_inclusion (meets_criteria yes))
    =>
    (println "___________")
    (println "Symptoms are consistent with both endometriosis and presence of concomitant disease. [recommendation]")
    (println "___________")
    (store_result endo_1 other_1)
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_INCLUSION)

DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_EXCLUSION = """
(defrule endo-and-concoomitant-inclusion
    (endometriosis_inclusion (meets_criteria no))
    (concomitant_disease_inclusion (meets_criteria no))
    =>
    (println "___________")
    (println "Symptoms are consistent with neither endometriosis nor concomitant diseases that we screened for (IBS, adenomyosis, others). [recommendation]")
    (println "___________")
    (store_result endo_0 other_0)
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_EXCLUSION)

DEFRULE_ONLY_ENDOMETRIOSIS_INCLUSION = """
(defrule only-endo-inclusion
    (endometriosis_inclusion (meets_criteria yes))
    (concomitant_disease_inclusion (meets_criteria no))
    =>
    (println "___________")
    (println "Patient has symptom(s) consistent only with endometriosis and not additional diseases")
    (println "___________")
    (store_result endo_1 other_0)
)
"""
env.build(DEFRULE_ONLY_ENDOMETRIOSIS_INCLUSION)

DEFRULE_ONLY_ENDOMETRIOSIS_EXCLUSION = """
(defrule only-endo-exclusion
    (endometriosis_inclusion (meets_criteria no))
    (concomitant_disease_inclusion (meets_criteria yes))
    =>
    (println "___________")
    (println "Patient has symptom(s) consistent only with non-endo diseases")
    (println "___________")
    (store_result endo_0 other_1)
)
"""
env.build(DEFRULE_ONLY_ENDOMETRIOSIS_EXCLUSION)

# Execution
You should use the cells below to test the system behavior while implementing the `defrules` in this assignment. 

In [ ]:
env.reset();
env.run();

In [ ]:
print_facts(env)

## Evaluating Expert System Performance

In [ ]:
# three outputs
# calculate accuracy, PPV, NPV for each case

# and ablation analysis

# input data path: /project/ssverma_shared/projects/Endometriosis/Endo_RuleBased_Phenotyping/symptom_pulls/Pheno/PMBB_2.3_pheno_covars.csv